# Scraping  Information about Cars from Kelly Blue Book. 

In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
url  = 'https://www.kbb.com/car-finder/2/?vehicletypes=luxury&morefilters=topratedexpert'
r = requests.get(url)
print(r) 

<Response [200]>


In [3]:
soup = BeautifulSoup(r.content,'lxml')
#print(soup.prettify())

In [4]:
car_div = soup.find_all('a',class_='css-y4ixr8 e5qxycd0')

In [ ]:
#car_div

In [5]:
name = []
startingPricce=[]
combFule = []
expertRating = []
cusmtRating = []

In [6]:
for container in car_div:
    cname = container.h2.text 
    name.append(cname)
    
    price = container.h3.text
    startingPricce.append(price)

    fl = container.find_all('div',class_='css-1oonbm0 eytv4cm0')
    fuel = fl[1].text
    combFule.append(fuel)
    
    expRating = container.find_all('div',class_='css-x9skgx-NumericRating e149jidm1')
    expRat = expRating[0].text
    expertRating.append(expRat)
    
    custRating = expRating[1].text
    cusmtRating.append(custRating)
   

In [7]:
df_cars = pd.DataFrame({'Car_Name':name,
                        'Starting_Price_USD':startingPricce,
                        'MPG_Combined_fuel_economy':combFule,
                        'Expert_Rating':expertRating,
                        'Customer_Rating':cusmtRating})

In [9]:
df_cars.head(10)

,Car_Name,Starting_Price_USD,MPG_Combined_fuel_economy,Expert_Rating,Customer_Rating
0,Certified 2019 Audi Q3 SUV,"$29,617",Combined Fuel Economy22 MPG,4.8,3.6
1,Certified 2018 Audi Q5 SUV,"$30,313",Combined Fuel Economy25 MPG,4.8,4.8
2,Certified 2018 Audi A5 Coupe,"$30,856",Combined Fuel Economy27 MPG,4.8,4.5
3,Certified 2017 Audi Q7 SUV,"$32,894",Combined Fuel Economy22 MPG,4.8,4.5
4,Certified 2019 Audi A5 Sedan,"$34,313",Combined Fuel Economy27 MPG,4.8,4.5
5,Certified 2019 Lexus ES Sedan,"$36,205",Combined Fuel Economy26 MPG,4.8,3.6
6,Certified 2020 Lexus ES Sedan,"$39,256",Combined Fuel Economy26 MPG,4.8,3.6
7,Certified 2018 Audi S5 Coupe,"$43,483",Combined Fuel Economy24 MPG,4.8,5
8,Certified 2018 Audi SQ5 SUV,"$43,740",Combined Fuel Economy21 MPG,4.8,4.8
9,Certified 2017 Mercedes‑Benz Mercedes‑AMG E‑Cl...,"$48,129",Combined Fuel Economy21 MPG,4.8,4.2


#### Now lets clean up the data 

In [ ]:
df_cars.head(1)

Remove Combined Fuel Economy that is before the number and also MPG 

In [10]:
df_cars['MPG_Combined_fuel_economy']=df_cars['MPG_Combined_fuel_economy'].str.extract('(\d+)').astype(int)

In [11]:
df_cars.head(1)

,Car_Name,Starting_Price_USD,MPG_Combined_fuel_economy,Expert_Rating,Customer_Rating
0,Certified 2019 Audi Q3 SUV,"$29,617",22,4.8,3.6


Remove comma in the dollar amount 

In [12]:
df_cars['Starting_Price_USD'] = df_cars['Starting_Price_USD'].str.replace(',','')

In [13]:
df_cars.head(1)

,Car_Name,Starting_Price_USD,MPG_Combined_fuel_economy,Expert_Rating,Customer_Rating
0,Certified 2019 Audi Q3 SUV,$29617,22,4.8,3.6


Remove the Dollar sign that is before the dollar amount 

In [14]:
df_cars['Starting_Price_USD'] = df_cars['Starting_Price_USD'].str.extract('(\d+)').astype(int)

In [16]:
df_cars.head(20)

,Car_Name,Starting_Price_USD,MPG_Combined_fuel_economy,Expert_Rating,Customer_Rating
0,Certified 2019 Audi Q3 SUV,29617,22,4.8,3.6
1,Certified 2018 Audi Q5 SUV,30313,25,4.8,4.8
2,Certified 2018 Audi A5 Coupe,30856,27,4.8,4.5
3,Certified 2017 Audi Q7 SUV,32894,22,4.8,4.5
4,Certified 2019 Audi A5 Sedan,34313,27,4.8,4.5
5,Certified 2019 Lexus ES Sedan,36205,26,4.8,3.6
6,Certified 2020 Lexus ES Sedan,39256,26,4.8,3.6
7,Certified 2018 Audi S5 Coupe,43483,24,4.8,5
8,Certified 2018 Audi SQ5 SUV,43740,21,4.8,4.8
9,Certified 2017 Mercedes‑Benz Mercedes‑AMG E‑Cl...,48129,21,4.8,4.2


### That is just scrapping one page. Lets now scrape information from multiple pages. 

In [ ]:
from time import sleep
from random import randint

In [ ]:


name = []
startingPricce=[]
combFule = []
expertRating = []
cusmtRating = []

kbb_pages = np.arange(1,61,10)

for page in kbb_pages:
    page = requests.get('https://www.kbb.com/car-finder/' + str(page) + '?vehicletypes=luxury&morefilters=topratedexpert')
    soup = BeautifulSoup(page.content,'lxml')
    car_div = soup.find_all('a',class_='css-y4ixr8 e5qxycd0')
    
    sleep(randint(2,10)) 
    
    for container in car_div:
        cname = container.h2.text 
        name.append(cname)

        price = container.h3.text
        startingPricce.append(price)

        fl = container.find_all('div',class_='css-1oonbm0 eytv4cm0')
        fuel = fl[1].text
        combFule.append(fuel)

        expRating = container.find_all('div',class_='css-x9skgx-NumericRating e149jidm1')
        expRat = expRating[0].text
        expertRating.append(expRat)

        custRating = expRating[1].text
        cusmtRating.append(custRating)
        
df_cars = pd.DataFrame({'Car_Name':name,
                        'Starting_Price_USD':startingPricce,
                        'MPG_Combined_fuel_economy':combFule,
                        'Expert_Rating':expertRating})
    

    

In [ ]:
df_cars.shape

In [ ]:
df_cars.head(20)

### Lets clean up the data now 

Remove $ and comma from price 

In [ ]:
df_cars['Starting_Price_USD'] = df_cars['Starting_Price_USD'].str.replace(',','')
df_cars['Starting_Price_USD'] = df_cars['Starting_Price_USD'].str.extract('(\d+)').astype(int)

In [ ]:
df_cars['MPG_Combined_fuel_economy'] = df_cars['MPG_Combined_fuel_economy'].str.extract('(\d+)').astype(int)

In [ ]:
df_cars.head(40)

In [ ]:
df_cars.info()

In [ ]:
df_cars['Expert_Rating']=df_cars['Expert_Rating'].astype(float) 

In [ ]:
df_cars['Customer_Rating'] = df_cars['Customer_Rating'].astype(float)

In [ ]:
df_cars.info() 

### Download the file to excel csv 

In [ ]:
df_cars.to_csv('LuxuryCars.csv')